# Save static STAC catalog for browsing

In [1]:
import pystac
import hyp3_sdk as sdk
import geopandas as gpd
import pandas as pd

import concurrent.futures
import utils # STAC functions

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# ITEMs for ASF HYP3
# Look up fileURLs from HYP3 SDK
hyp3 = sdk.HyP3() 
myjobs = hyp3.find_jobs(job_type='INSAR_ISCE_BURST')

In [4]:
# interesting, first time one failed
df = pd.DataFrame([x.to_dict() for x in myjobs])
df.status_code.value_counts()

status_code
SUCCEEDED    483
FAILED         1
Name: count, dtype: int64

In [5]:
df['refdate'] = pd.to_datetime(df['name'].str[15:23], format='%Y%m%d', errors='coerce')
df = df[df['refdate'].notna()] # drop rows with NaT

df['secdate'] = pd.to_datetime(df['name'].str[24:32], format='%Y%m%d', errors='coerce')
df['timespan'] = df.secdate - df.refdate

df = df.sort_values(by='refdate').reset_index()

In [6]:
subset = df[df['name'].str.startswith('064')]
subset = subset[subset.status_code == 'SUCCEEDED']
#subset = subset[subset.request_time.str.startswith('2023-12-08')]
print(len(subset))

279


In [7]:
# For timeseries, we need n_interferograms > n_dates
dates = pd.unique(subset[['refdate', 'secdate']].values.ravel())
print(f'{len(subset)} Interferograms')
print(f'{len(dates)} Dates')

279 Interferograms
139 Dates


In [8]:
subset.iloc[0].files

[{'filename': 'S1_135602_IW1_20190111_20190312_VV_INT20_8818.zip',
  's3': {'bucket': 'hyp3-edc-prod-contentbucket-1fv14ed36ifj6',
   'key': '03620eef-e5dd-40b1-9531-a07e1c57772d/S1_135602_IW1_20190111_20190312_VV_INT20_8818.zip'},
  'size': 93802111,
  'url': 'https://d3gm2hf49xd6jj.cloudfront.net/03620eef-e5dd-40b1-9531-a07e1c57772d/S1_135602_IW1_20190111_20190312_VV_INT20_8818.zip'}]

## Single ItemCollection

In [ ]:
# %%time 

# args = [job for i,job in subset.iterrows()]

# # NOTE: cleaner to use dask dataframe here!
# with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
#     results = executor.map(hyp32stac, args)

In [ ]:
# items = list(results)

In [ ]:
# Save as ItemCollection
# pystac.ItemCollection(items).save_object()

In [9]:
# Load previously saved items
items = pystac.ItemCollection.from_file('A64-itemcollection.geojson')

In [10]:
item = items[0]
item

<Item id=064_135602_IW1_20190111_20190312_5rlks>

## Static catalog

For now flat catalog of items (w/ burstId subcatalog)
NOTE: currently does not append, just re-writes

In [13]:
CATALOG_ID = 'uw-subsidence'

catalog = pystac.Catalog(id=CATALOG_ID,
                         description='https://github.com/relativeorbit/AGU2023')

collection = utils.create_collection(CATALOG_ID)
catalog.add_child(collection)
collection.add_items(items)
# NOTE: subcatalogs based on STAC properties and datetime pieces
catalog.generate_subcatalogs(template='${burstId}/${year}')
catalog.normalize_hrefs('./')
catalog.validate()
catalog.save(catalog_type=pystac.CatalogType.RELATIVE_PUBLISHED)

NameError: name 'save_item_collection' is not defined